In [1]:
from collections import namedtuple
import re

import pandas as pd
import numpy as np

In [2]:
max_gene = 65535

In [3]:
def generate_mutant(gene, gender = None, length = 9):
    
    mutant_factor = 0.7
    factors = [f'+{np.random.randint(1, 5)}', f'-{np.random.randint(1, 5)}', 
               f'*{np.random.randint(1, 5)}', f'/{np.random.randint(1, 5)}']
    
    normal = np.random.normal()
    
    if gender == 'M':
        
        if abs(normal) > 0.5:
            gene[0] *= 0.1
            gene[1] *= 1.2
            gene[2] *= 0.9
            gene[3] *= 0.3
            gene[4] *= 0.4
            gene[5] *= 0.2
            gene[6] *= 0.5
            gene[7] *= 0.8
            gene[8] *= 1.3
            
    elif gender == 'F':
        
        if abs(normal) > 0.5:
            gene[0] *= 1.3
            gene[1] *= 0.4
            gene[2] *= 0.3
            gene[3] *= 1.1
            gene[4] *= 0.4
            gene[5] *= 0.7
            gene[6] *= 0.6
            gene[7] *= 0.8
            gene[8] *= 1.3
    
    else: pass
    
    if abs(normal) > mutant_factor:
        
        for _ in range(length // 3):
            factor    = factors[np.random.randint(4)]
            idx       = np.random.randint(length)
            gene[idx] = factor
            
    return gene


def generate_new_ind(male, female):
    
    symbols  = ['+', '-', '*', '/']
    new_gene = []
    for m, f in zip(male[:-1], female[:-1]):
        condition = [m[0] in symbols, f[0] in symbols]
        if any(condition):
            
            if all(condition):
                m = re.sub('[^0-9]', '', m)
                f = re.sub('[^0-9]', '', f)
                
                new_gene.append(max(float(m), float(f)))
                
            else:
                
                gene = int(eval(f + m)) if m[0] in symbols else int(eval(m + f))
                new_gene.append(np.clip(gene, 0, max_gene))
                
        else: new_gene.append(max(float(m), float(f)))
       
    gender   = 'M' if abs(np.random.normal()) > 0.63 else 'F'    
    new_gene = generate_mutant(new_gene, gender, length = len(new_gene))
    
    new_gene.append(gender)
    return list(map(str, new_gene))

def generate_gene(length = 9):
    
    gene = list(map(str, np.random.randint(15, size = 9)))
    gene = generate_mutant(gene, length = length)
    
    return gene

def generate_parents():
    init_inds = {
                'f1' : generate_gene() + ['F'],
                'm1' : generate_gene() + ['M'],
            }
        
    return init_inds

In [4]:
people = generate_parents()
people

{'f1': ['2', '5', '6', '6', '4', '10', '0', '4', '8', 'F'],
 'm1': ['1', '10', '2', '11', '3', '3', '5', '8', '7', 'M']}

In [5]:
cnt         = 2
family_tree = {}

while True:
    
    males   = [people[gen] for gen in people.keys() if 'm' in gen]
    females = [people[gen] for gen in people.keys() if 'f' in gen]
    
    male_idx   = np.random.randint(len(males))
    female_idx = np.random.randint(len(females))
    
    male       = males[male_idx]
    female     = females[female_idx]
    
    child      = generate_new_ind(male, female)
    gender     = child[-1].lower()
    

    child_idx   = f'{gender}{cnt}'
    people[child_idx] = child
    
    males      = [key for key in people.keys() if 'm' in key]
    females    = [key for key in people.keys() if 'f' in key]

    father     = males[male_idx]
    mother     = females[female_idx]
    
    family_tree[cnt] = {'father' : father, 'mother' : mother, 'child' : child_idx}
    
    if len(people.keys()) > 20: 
        people.pop(list(people.keys())[0])
        
    
    cnt += 1
    if cnt > 1e+4: break

In [6]:
family_df = pd.DataFrame(family_tree).T
family_df = pd.DataFrame(family_df)

In [7]:
people[list(people.keys())[-1]]

['335446.749867844',
 '94370.4',
 '4.0',
 '26432.0',
 '4.0',
 '2.8',
 '3',
 '11331.0',
 '65535',
 'M']